In [19]:
#Practice Word2vec from https://adventuresinmachinelearning.com/word2vec-tutorial-tensorflow/

import urllib.request
import collections
import math
import os
import random
import zipfile
import datetime as dt

import numpy as np
import tensorflow as tf

In [20]:
def download_text_to_file_from_website(filename, url, exp_bytes):
    if not os.path.exists(filename):
        filename, _ = urllib.request.urlretrieve(url+filename, filename)
    info = os.stat(filename)
    if info.st_size == exp_bytes:
        print("Found and Verified", filename)
    else:
        print(info.st_size)
        raise Exception("Failed to verify")
    return filename

In [21]:
def read_data_from_file(filename):
    with zipfile.ZipFile(filename) as f:
        data = tf.compat.as_str(f.read(f.namelist()[0])).split()
    return data

In [22]:
def build_dataset(words, n_words):
    count = [['UNK',-1]]
    count.extend(collections.Counter(words).most_common(n_words-1))
    
    dictionary = dict()
    
    for word,_ in count:
        dictionary[word] = len(dictionary)
        
    data = list()
    unk_count = 0
    for word in words:
        if word in dictionary:
            index = dictionary[word]
        else:
            index = 0
            unk_count += 1
        data.append(index)
    
    count[0][1] = unk_count
    reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    
    return data, count, dictionary, reverse_dictionary

In [23]:
def collect_data(vocabulary_size=10000):
    url = "http://mattmahoney.net/dc/"
    filename = download_text_to_file_from_website('text8.zip', url, 31344016)
    vocabulary = read_data_from_file(filename)
    print(vocabulary[:10])
    data, count, dictionary, reverse_dictionary = build_dataset(vocabulary,
                                                               vocabulary_size)
    del vocabulary
    return data, count, dictionary, reverse_dictionary

In [24]:
data_index = 0


In [25]:
def generate_batch(data, batch_size, num_skips, skip_window):
    global data_index
    assert batch_size % num_skips == 0
    assert num_skips <= 2 * skip_window
    
    batch = np.ndarray(shape=(batch_size), dtype=np.int32)
    context = np.ndarray(shape=(batch_size,1), dtype=np.int32)
    
    span = 2 * skip_window + 1
    
    buffer = collections.deque(maxlen = span)
    
    for _ in range(span):
        buffer.append(data[data_index])
        data_index = (data_index+1)%len(data)
        
    for i in range(batch_size // num_skips):
        target = skip_window
        targets_to_avoid = [skip_window]
        for j in range(num_skips):
            while target in targets_to_avoid:
                target = random.randint(0, span-1)
            targets_to_avoid.append(target)
            batch[i*num_skips+j]=buffer[skip_window]
            context[i*num_skips+j,0]=buffer[target]
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)
        
    data_index = (data_index+len(data) - span) % len(data)
    return batch,context

In [26]:
vocabulary_size = 10000
data, count, dictionary, reverse_dictionary = collect_data()

Found and Verified text8.zip
['anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse', 'first', 'used', 'against']


In [27]:
batch_size = 128

#Feature Space of the embedding vector 
embedding_size = 300

#How many words in the window
skip_window = 2

num_skips = 2

In [28]:
valid_size = 16     # Random set of words to evaluate similarity on.
valid_window = 100  # Only pick dev samples in the head of the distribution.
valid_examples = np.random.choice(valid_window, valid_size, replace=False)
num_sampled = 64 

In [29]:
graph = tf.Graph()


In [30]:
with graph.as_default():
    
    #Input Data
    
    train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
    train_context = tf.placeholder(tf.int32, shape=[batch_size, 1])
    valid_dataset = tf.constant(valid_examples, dtype=tf.int32)
    
    embeddings = tf.Variable(
          tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
    embed = tf.nn.embedding_lookup(embeddings, train_inputs)

    
    weights = tf.Variable(
        tf.truncated_normal([embedding_size, vocabulary_size],
        stddev=1.0 / math.sqrt(embedding_size)))
    
    biases = tf.Variable(tf.zeros([vocabulary_size]))
    
    hidden_out = tf.transpose(tf.matmul(tf.transpose(weights), tf.transpose(embed))) + biases
 

    train_one_hot = tf.one_hot(train_context, vocabulary_size)
    
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=hidden_out, labels=train_one_hot))

    optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(cross_entropy)
    
    norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
    normalized_embeddings = embeddings / norm
    valid_embeddings = tf.nn.embedding_lookup(
      normalized_embeddings, valid_dataset)
    similarity = tf.matmul(
      valid_embeddings, normalized_embeddings, transpose_b=True)
    
    init = tf.global_variables_initializer()


In [31]:
def run(graph, num_steps):
    with tf.Session(graph=graph) as session:
        init.run()
        
        average_loss = 0
        for step in range(num_steps):
            batch_inputs, batch_context = generate_batch(data,
                batch_size, num_skips, skip_window)
            feed_dict = {train_inputs: batch_inputs, train_context: batch_context}

            _, loss_val = session.run([optimizer, cross_entropy], feed_dict=feed_dict)
            average_loss += loss_val
            
            
            if step % 2000 == 0:
                if step > 0:
                    average_loss /= 2000
                  # The average loss is an estimate of the loss over the last 2000 batches.
                print('Average loss at step ', step, ': ', average_loss)
                average_loss = 0
                
                
            if step % 10000 == 0:
                sim = similarity.eval()
                for i in range(valid_size):
                    valid_word = reverse_dictionary[valid_examples[i]]
                    top_k = 8  # number of nearest neighbors
                    nearest = (-sim[i, :]).argsort()[1:top_k + 1]
                    log_str = 'Nearest to %s:' % valid_word
                    for k in range(top_k):
                        close_word = reverse_dictionary[nearest[k]]
                        log_str = '%s %s,' % (log_str, close_word)
                        print(log_str)
                        
        final_embeddings = normalized_embeddings.eval()

In [ ]:
num_steps = 300
softmax_start_time = dt.datetime.now()
run(graph, num_steps=num_steps)
softmax_end_time = dt.datetime.now()
print("Softmax method took {} minutes to run 100 iterations".format((softmax_end_time-softmax_start_time).total_seconds()))


In [ ]:
with graph.as_default():

    # Construct the variables for the NCE loss
    nce_weights = tf.Variable(
        tf.truncated_normal([vocabulary_size, embedding_size],
                            stddev=1.0 / math.sqrt(embedding_size)))
    nce_biases = tf.Variable(tf.zeros([vocabulary_size]))

    nce_loss = tf.reduce_mean(
        tf.nn.nce_loss(weights=nce_weights,
                       biases=nce_biases,
                       labels=train_context,
                       inputs=embed,
                       num_sampled=num_sampled,
                       num_classes=vocabulary_size))

    optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(nce_loss)

    # Add variable initializer.
    init = tf.global_variables_initializer()

num_steps = 400
nce_start_time = dt.datetime.now()
run(graph, num_steps)
nce_end_time = dt.datetime.now()
print("NCE method took {} minutes to run 100 iterations".format((nce_end_time-nce_start_time).total_seconds()))

Average loss at step  0 :  9.352001190185547
Nearest to four: medical,
Nearest to four: medical, practice,
Nearest to four: medical, practice, circle,
Nearest to four: medical, practice, circle, departure,
Nearest to four: medical, practice, circle, departure, improved,
Nearest to four: medical, practice, circle, departure, improved, consent,
Nearest to four: medical, practice, circle, departure, improved, consent, volume,
Nearest to four: medical, practice, circle, departure, improved, consent, volume, cue,
Nearest to he: belt,
Nearest to he: belt, confusing,
Nearest to he: belt, confusing, morgan,
Nearest to he: belt, confusing, morgan, schemes,
Nearest to he: belt, confusing, morgan, schemes, ink,
Nearest to he: belt, confusing, morgan, schemes, ink, skills,
Nearest to he: belt, confusing, morgan, schemes, ink, skills, banking,
Nearest to he: belt, confusing, morgan, schemes, ink, skills, banking, happen,
Nearest to in: manga,
Nearest to in: manga, circa,
Nearest to in: manga, circa